In [15]:
import numpy as np
from astropy.io import fits
from astropy.wcs import WCS
from astropy.coordinates import SkyCoord
from astropy import units as u
import matplotlib.pyplot as plt
from scipy import stats 
from astroquery.mast import Catalogs
import math
from astroquery.vizier import Vizier
from astropy.io import ascii
from astropy.table import Column
import pickle

In [19]:
data = ascii.read("test_residuals_large.txt")
data

col1,col2,col3,col4,col5,col6,col7
float64,float64,float64,float64,float64,float64,float64
151.969292,1.549148,16.35691,16.3501,0.00681,16.342706,-0.007394
152.027884,1.55079,16.88637,16.8353,0.05107,16.896663,0.061363
151.983179,1.555922,17.48092,17.4708,0.01012,17.465545,-0.005255
152.020791,1.557273,18.45524,18.3986,0.05664,18.314801,-0.083799
151.979621,1.568498,17.35203,17.3558,-0.00377,17.325274,-0.030526
151.932363,1.573478,16.8575,16.8855,-0.028,16.872055,-0.013445
152.019911,1.577202,18.49584,18.5168,-0.02096,18.459923,-0.056877
151.990913,1.596397,16.84828,16.8579,-0.00962,16.818553,-0.039347
151.968135,1.598542,17.04578,17.0615,-0.01572,17.036542,-0.024958


In [20]:
largerespos=np.where(data ['col5']>0.08)
largeresneg=np.where(data ['col5']<-0.08)
print (largerespos,largeresneg)

largeresposap=np.where(data ['col7']>0.08)
largeresnegap=np.where(data ['col7']<-0.08)
print (largeresposap,largeresnegap)

largeresrapos=data['col1'][largerespos]
largeresdecpos=data['col2'][largerespos]

largeresraneg=data['col1'][largeresneg]
largeresdecneg=data['col2'][largeresneg]


def create_pixregionfile(x,y,regionname,color,coords='fk5'):
        with open(regionname, 'w') as f:
            f.write('global color={0} dashlist=8 3 width=2 font=\"helvetica 10 normal roman\" select=1 highlite=1 dash=0 fixed=0 edit=1 move=1 delete=1 include=1 source=1 \n'.format(color))
            f.write('%s \n'%coords)
            for star in range(len(x)):
                xval = x[star]
                yval = y[star]
                f.write('circle({ra},{dec},{radius}")\n'.format(ra=xval, dec=yval,radius=5))

    #     return (ra_wcs,dec_wcs,flux)
        f.close()
        
create_pixregionfile(largeresrapos,largeresdecpos,'largespos.reg','red')  
create_pixregionfile(largeresraneg,largeresdecneg,'largesneg.reg','green')    

(array([101, 228, 326, 399, 400, 403, 444]),) (array([ 98, 148]),)
(array([ 20,  45,  83, 152, 255, 316, 367, 421, 432, 434, 462]),) (array([  3,  37,  98, 145, 195, 206, 294, 446, 452, 454]),)


In [21]:
np.sort(data['col5'])

-1.09439
-0.08522
-0.0761
-0.07228
-0.07152
-0.06912
-0.06658
-0.06648
-0.06331
-0.06025
-0.0602
